<a href="https://colab.research.google.com/github/kuanyshbakytuly/dataton/blob/main/checking_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms,models,datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torch import optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import cv2, glob, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
import pandas as pd

In [18]:
test_data = "/content/test_imgs"

In [19]:
def get_model():
    #Returning the pretrained model - ResNET34
    model = models.resnet34(pretrained=True)

    #Freezing all parameters
    for param in model.parameters():
        param.requires_grad = False

    model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    model.fc = nn.Sequential(nn.Flatten(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 1),
    nn.Sigmoid())

    #loss_function - Binary Cross Entropy
    #optimizer - Adam
    #Recommended by https://www.mdpi.com/1424-8220/22/5/1701 for Fire Classifying
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= 1e-3)
    return model.to(device), loss_fn, optimizer

In [20]:
@torch.no_grad()
def accuracy_testing(x, model):
    #Evaluating the model without training it
    prediction = model(x)

    #if 0<prediction<0.5 => return False
    #else True
    is_correct = (prediction > 0.5)
    return is_correct.cpu().numpy().tolist()

In [ ]:
model, x, y = get_model()
model.load_state_dict(torch.load('/content/resnet34.pth'))
model.eval()

In [22]:
from torch.utils.data import DataLoader, Dataset
class Test(Dataset):
    #Providing the class that returns images

    def __init__(self, folder):
        #Initializing images from folder 
        images_in_folder =  glob(folder + '/*.jpg')
        self.images = [image_path for image_path in images_in_folder]
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

    def __len__(self): 
        return len(self.images)

    def __getitem__(self, i):
        #Getting images[i] and its target
        image_path = self.images[i]
        im = (cv2.imread(image_path)[:,:,::-1])
        im = cv2.resize(im, (224,224))
        im = torch.tensor(im/255)
        im = im.permute(2,0,1)
        im = self.normalize(im) 

        return im.float().to(device),  image_path.split("/")[-1] 

In [23]:
test = Test(test_data)
tst_dl = DataLoader(test, batch_size=1, shuffle=True, drop_last = True)

In [24]:
import csv
dict_test = {}
for ix, batch in enumerate(iter(tst_dl)):
    x, image_name = batch
    test_is_correct = accuracy_testing(x, model)
    dict_test[image_name[0]] = int(test_is_correct[0][0])

In [25]:
(pd.DataFrame.from_dict(data=dict_test, orient='index')
   .to_csv('/content/answer.csv', header=False))